In [1]:
#Import modules

import pandas as pd
# from IPython.display import display, HTML
# from numpy import log10, nan, inf, random, clip, arange

# path = './../export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'

# display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
%matplotlib inline

import dask.dataframe as dd
from dask.diagnostics import ProgressBar

from numpy import arange, log10

from functions import chunk, agg, finalize
tunique = dd.Aggregation('tunique', chunk, agg,finalize)
first = dd.Aggregation('first', chunk, agg,finalize)

In [2]:
# dd.read_csv('2014-*.csv')

# drive_path = './../export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'
drive_path = './../../../../../media/miglesia/Elements/export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'

n_bins = 20
df_list = n_bins * [ 0 ]

for dataset_i in range(n_bins):
    df_list[dataset_i] = dd.read_csv(drive_path+'/samplings/YMxpb_size20'+str(dataset_i).zfill(2)+'.csv',
                                    usecols = ['YEAR','ID','VAT','CN ID 8', 'VART_sum'])
    
    

In [3]:
data = dd.concat(df_list[8:])
data = data.groupby(['YEAR','ID', 'VAT', 'CN ID 8'])['VART_sum'].sum().reset_index()

In [4]:
with ProgressBar():
    out = data.compute()

data = dd.from_pandas(out, npartitions = 500)

[########################################] | 100% Completed |  1min 24.8s


In [ ]:
df = data.compute()

In [ ]:
grouped = df.groupby(['YEAR', 'ID'])
margins_df = grouped.agg({'VAT': 'nunique', 'CN ID 8': 'nunique'}).add_suffix('_nunique')

margins_df['density'] = grouped.count().iloc[:, 1] / (margins_df['CN ID 8_nunique']*margins_df['VAT_nunique'])
margins_df['avg_x'] = grouped['VART_sum'].mean()
margins_df['sum_x'] = grouped['VART_sum'].sum()

In [ ]:
margins_table = margins_df.unstack().swaplevel(axis = 1).sort_index(axis = 1)
margins_table

In [ ]:
log_growth_margins_table = log10(margins_table).diff()
log_growth_margins_table

In [ ]:
# pd.DataFrame(log_growth_margins_table.mean()).unstack().sort_values(by = (0, 'sum_x'))
pd.DataFrame(log_growth_margins_table.mean()).reset_index().groupby('level_1')[0].describe()

In [ ]:
# firm_sizes = log10(data.groupby(['ID', 'YEAR'])['VART_sum'].sum().reset_index().groupby('ID')[['VART_sum']].mean().compute())